In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

list_1 = []
for sqrt_value in generator:
  list_1.append(sqrt_value)

print(sum(list_1))

8.382332347441762


In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

list_1 = []
for sqrt_value in generator:
  list_1.append(sqrt_value)

print(list_1[12])
print(list_1[-1])

3.605551275463989
3.605551275463989


In [3]:
%%capture
!pip install dlt[duckdb]

In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data_1 = []
for person in people_1():
    data_1.append(person)

In [6]:
import dlt
pipeline = dlt.pipeline(pipeline_name="hw_data",
						destination='duckdb',
						dataset_name='demog_data')

In [7]:
demog_1 = pipeline.run(data_1,
                    table_name="demographics",
                    write_disposition="replace")

In [8]:
print(demog_1)

Pipeline hw_data load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset demog_data
The duckdb destination used duckdb:////content/hw_data.duckdb location to store data
Load package 1708073641.1542614 is LOADED and contains no failed jobs


In [9]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data_2 = []
for person in people_2():
    data_2.append(person)

In [10]:
demog_1 = pipeline.run(data_2,
                    table_name="demographics",
                    write_disposition="append")

In [11]:
print(demog_1)

Pipeline hw_data load step completed in 0.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset demog_data
The duckdb destination used duckdb:////content/hw_data.duckdb location to store data
Load package 1708073641.9838643 is LOADED and contains no failed jobs


In [12]:
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT "ID", "Name", "Age", "City", "Occupation" FROM demographics'
    ) as table:
        Demographics = table.df()


In [13]:
Demographics

,id,name,age,city,occupation
0,1,Person_1,26,City_A,None
1,2,Person_2,27,City_A,None
2,3,Person_3,28,City_A,None
3,4,Person_4,29,City_A,None
4,5,Person_5,30,City_A,None
5,3,Person_3,33,City_B,Job_3
6,4,Person_4,34,City_B,Job_4
7,5,Person_5,35,City_B,Job_5
8,6,Person_6,36,City_B,Job_6
9,7,Person_7,37,City_B,Job_7


In [14]:
Demographics["age"].sum()

353

In [15]:
demog_1 = pipeline.run(data_1,
                    table_name="demographics_2",
                    write_disposition="replace")

In [16]:
demog_1 = pipeline.run(data_2,
                    table_name="demographics_2",
                    write_disposition="merge",
                       primary_key="ID")

In [17]:
with pipeline.sql_client() as client:
    with client.execute_query(
        'SELECT "ID", "Name", "Age", "City", "Occupation" FROM demographics_2'
    ) as table:
        Demographics_2 = table.df()

In [18]:
Demographics_2

,id,name,age,city,occupation
0,1,Person_1,26,City_A,None
1,2,Person_2,27,City_A,None
2,5,Person_5,35,City_B,Job_5
3,7,Person_7,37,City_B,Job_7
4,3,Person_3,33,City_B,Job_3
5,6,Person_6,36,City_B,Job_6
6,8,Person_8,38,City_B,Job_8
7,4,Person_4,34,City_B,Job_4


In [19]:
Demographics_2["age"].sum()

266